<a href="https://colab.research.google.com/github/preenet/961733-EnergyDataAnalytics/blob/master/9717333_Week_10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib
%load_ext autoreload
%autoreload 2 

Using matplotlib backend: agg


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Import dataset

In [0]:
data_path = '/content/drive/My Drive/Colab Notebooks/'
df = pd.read_excel(data_path+'ENB2012_data.xlsx')

In [5]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


## 2. Prepocessing  
Transformation for ordinal vars

In [0]:
from sklearn.preprocessing import MinMaxScaler

x5 = pd.get_dummies(df['X5'], prefix='X5', dtype= float)
df = pd.concat([df, x5], axis =1)

x6 = pd.get_dummies(df['X6'], prefix='X6', dtype= float)
df = pd.concat([df, x6], axis =1)

x8 = pd.get_dummies(df['X8'], prefix='X8', dtype= float)
df = pd.concat([df, x8], axis =1)

# now drop the original columns
df.drop(['X5'],axis=1, inplace=True)
df.drop(['X6'],axis=1, inplace=True)
df.drop(['X8'],axis=1, inplace=True)

In [7]:
df.head()

,X1,X2,X3,X4,X7,Y1,Y2,X5_3.5,X5_7.0,X6_2,X6_3,X6_4,X6_5,X8_0,X8_1,X8_2,X8_3,X8_4,X8_5
0,0.98,514.5,294.0,110.25,0.0,15.55,21.33,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.98,514.5,294.0,110.25,0.0,15.55,21.33,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.98,514.5,294.0,110.25,0.0,15.55,21.33,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.98,514.5,294.0,110.25,0.0,15.55,21.33,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.90,563.5,318.5,122.50,0.0,20.84,28.28,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# normalized with minmax algorithm
scaler = MinMaxScaler(feature_range=(-1,1))
normalized_df = scaler.fit_transform(df)


# Modeling
Class MLPClassifier implements a multi-layer perceptron (MLP) algorithm that trains using Backpropagation.

In [0]:
from sklearn import preprocessing
# we are going to predict the heating load 'y1'

Y = normalized_df[:, 5]
X = np.delete(normalized_df, np.s_[5:7], axis=1)

# normalize the data attributes


In [16]:
X.shape

(768, 17)

In [17]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

cv = KFold(n_splits = 10, shuffle = True, random_state =False)
cv_scores_test = []
testing_error = []

# We are going to try only three parems as follows:

# hidden_layer_sizestuple, length = n_layers - 2, default=(100,)

# WEKA: There are also wildcard values: 'a' = (attribs + classes) / 2, 'i' = attribs, 'o' = classes , 't' = attribs + classes.

# activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default=’relu’
# solver{‘lbfgs’, ‘sgd’, ‘adam’}, default=’adam’

# For small datasets, however, ‘lbfgs’ can converge faster and perform better.
mlr = MLPRegressor(hidden_layer_sizes = (35,), activation = 'relu', solver = 'lbfgs', learning_rate_init = 0.3, momentum = 0.2, max_iter = 1000,  random_state=0)
for train_index, test_index in cv.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  
  mlr.fit(X_train, y_train)
  y_test_pred = mlr.predict(X_test)      
  
  cv_scores_test = mean_absolute_error(y_test, y_test_pred)
  print(cv_scores_test)
  testing_error.append(cv_scores_test)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.03580287964322695


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.040540732043575355


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.03766361313400908


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.0463908409027736
0.028228922568347296


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [18]:
print('crosss validation MAE. is:', np.mean(testing_error))

crosss validation MAE. is: 0.03632542722145618
